In [48]:
import numpy as np
import random as random

np.set_printoptions(threshold=np.inf)

In [23]:
def addObjetives(R,objetives):
    for objetive in objetives:
        for i in range(num_fil**2):
            R[i][objetive] = 100 if (R[i][objetive]==0) else -1       
        R[objetive][objetive] = 100
    return R          

In [26]:
def inicializeRMatrix(numRows, numCols):
    return np.array([-1 for _ in range((numRows**2)*(numCols**2))]).reshape(numRows**2, numCols**2)

In [51]:
size = int(input('¿Cuál es el tamaño del problema? '))
num_col = size
num_fil = size
states = (num_col*num_fil)**2

¿Cuál es el tamaño del problema?  5


In [52]:
objetivo = int(input('¿Cuál es el estado objetivo?'))

¿Cuál es el estado objetivo? 8


# Cálculo del tablero

Inicializamos la Matriz tablero primero. Para ello, según hemos observado en el ejemplo, lo valores del tablero se incrementan por lo que hemos denominado __capas__:


    -Capa 0: 
            [0]
    -Capa 1: 
            [0  3]
            [1  2]
    -Capa 2:
            [0  3  8]
            [1  2  7]
            [4  5  6]
    -Capa 3: 
            [ 0   3   8  15]
            [ 1   2   7  14]
            [ 4   5   6  13]
            [ 9  10  11  12]
    -Capa n:
            [ 0      3      8     15    .  (n^2)+2n  ]
            [ 1      2      7     14    .  (n^2)+2n-1]
            [ 4      5      6     13    .  (n^2)+2n-2]
            [ 9     10     11     12    .  (n^2)+2n-3]
              .      .      .      .    .      .
            [n^2 (n^2)+1 (n^2)+2 (n^2)+3.    (n^2)+n ]

Para su simplificación, hemos decidido cambiar la orientación propuesta en el enunciado del problema. Los resultados obtenidos en la matriz Q no van a variar de ninguna forma, lo único es que la matriz obtenida estaría desplazada __90º__ respecto de la original.

Para modular el código hemos separado la función que ejecuta el bucle __n__ veces (n = número de capas que queramos en la matriz) 

In [53]:
def createMatrix(m,tam):
    layer=0
    for i in range(tam):
        [m,layer]=addLayerToMatrix(m,i,layer)
    return m

Y la función que añade una capa más a la matriz

In [54]:
def addLayerToMatrix(matrix, layer, count):
    for i in range(layer):
        
        matrix[layer][i] = int(count)
        count=count+1
        
    for i in range(layer+1):
        
        matrix[layer-i][layer] = int(count)
        count=count+1
    
    return [matrix, count]

De ahí procedemos a crear el tablero deseado según el valor que hayamos introducido al principio del documento 

In [55]:
zeroMatrix= np.zeros((num_fil,num_col))
tablero = createMatrix(zeroMatrix,size).astype(int)
print(tablero)

[[ 0  3  8 15 24]
 [ 1  2  7 14 23]
 [ 4  5  6 13 22]
 [ 9 10 11 12 21]
 [16 17 18 19 20]]


# Cálculo de la matriz R

Una vez creado el tablero procederemos al calculo de la matriz R. Para ello nos basaremos en las posiciones de la matriz tablero. En esta ocasión los únicos movimientos válidos son los movimientos perpendiculares y diagonales. 

Para que la matriz obtenida no tenga en cuenta los espacios fuera del tablero, tenemos que controlar los movimientos en los bordes del tablero. Dicho de otra forma controlamos cuando cualquiera de los índices del bucle se encuentra en la posición 0 o en la posición máxima menos 1.

Cuando los índices se encuentren entre los valores mencionados en el párrafo anterior no existe restricción, menos que no puede ir a sí mismo. Esto último lo hemos decidido para que el entrenamiento de la matriz Q se ejecute de la forma más rápido posible.

In [56]:
#Este método obtiene automáticamente la Matriz R a partir del tablero (m) introducido por el usuario
def obtainRMatrix(m):
    Rsize = size**2
    rMatrix = inicializeRMatrix(num_col,num_fil)
    res=[]
    for i in range(num_fil):
        for j in range(num_col):
            n0 = m[i][j]
            if 0<i<num_fil-1:
                rMatrix[m[i][j]][m[i-1][j]]=0
                rMatrix[m[i][j]][m[i+1][j]]=0
                
                if(0<j<num_col-1):
                    rMatrix[m[i][j]][m[i-1][j-1]]=0
                    rMatrix[m[i][j]][m[i][j-1]]=0
                    rMatrix[m[i][j]][m[i-1][j+1]]=0
                    rMatrix[m[i][j]][m[i+1][j-1]]=0
                    rMatrix[m[i][j]][m[i+1][j+1]]=0
                    rMatrix[m[i][j]][m[i][j+1]]=0
                    
                if(j==0):
                    rMatrix[m[i][j]][m[i-1][j+1]]=0
                    rMatrix[m[i][j]][m[i][j+1]]=0
                    rMatrix[m[i][j]][m[i+1][j+1]]=0
                    
                if(j==num_col-1):
                    rMatrix[m[i][j]][m[i-1][j-1]]=0
                    rMatrix[m[i][j]][m[i][j-1]]=0
                    rMatrix[m[i][j]][m[i+1][j-1]]=0  
                    
            elif(i==0):
                rMatrix[m[i][j]][m[i+1][j]]=0             
                
                if(j==0):
                    rMatrix[m[i][j]][m[i+1][j+1]]=0             
                    rMatrix[m[i][j]][m[i][j+1]]=0   
                    
                if(j==num_col-1):
                    rMatrix[m[i][j]][m[i+1][j-1]]=0  
                    rMatrix[m[i][j]][m[i][j-1]]=0 
                    
                if(0<j<num_col-1):
                    rMatrix[m[i][j]][m[i+1][j-1]]=0  
                    rMatrix[m[i][j]][m[i+1][j+1]]=0  
                    rMatrix[m[i][j]][m[i][j+1]]=0  
                    rMatrix[m[i][j]][m[i][j-1]]=0  
                    
            elif i==num_fil-1:
                rMatrix[m[i][j]][m[i-1][j]]=0  
                if(j==0):
                    rMatrix[m[i][j]][m[i-1][j+1]]=0  
                    rMatrix[m[i][j]][m[i][j+1]]=0  
                if(j==num_col-1):
                    rMatrix[m[i][j]][m[i-1][j-1]]=0  
                    rMatrix[m[i][j]][m[i][j-1]]=0  
                if(0<j<num_col-1):
                    rMatrix[m[i][j]][m[i-1][j+1]]=0  
                    rMatrix[m[i][j]][m[i-1][j-1]]=0  
                    rMatrix[m[i][j]][m[i][j+1]]=0  
                    rMatrix[m[i][j]][m[i][j-1]]=0  
    return rMatrix

Una vez creada la función necesitamos añadirle el estado objetivo. Para ello hemos creado la funcion addOjetives. Aunque en este caso únicamente le añadimos sólo un objetivo, hemos implementado la opción de que se le pueda añadir más de un objetivo.

In [57]:
rMatrix=obtainRMatrix(tablero)
rMatrix=addObjetives(rMatrix,[objetivo])
print(rMatrix)

In [60]:
qMatrix = np.zeros((size**2,size**2)).astype(int)
print(qMatrix)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

Algoritmo de aprendizaje 
-----------------------------------------------------------

In [194]:
def acciones ( matriz, estado ):
    fila_estado = matriz [ estado , : ]
    acciones = [ ]
    for n in range ( len ( fila_estado ) ):
        if (fila_estado [ n ] != -1):
            acciones.append(n)
    return (acciones)

In [195]:
def maxQValue(Q, R, estado):
    #obtener los estados accesibles desde la matriz R
    rRow = R[estado]
    qRow = Q[estado]
    possibleStatesIndex = np.where(rRow != -1)
    #obtener los valores en la matriz Q
    qLista=qRow[possibleStatesIndex]
    #devolver el máximo
    maxValue=np.amax(qLista)
    return maxValue

In [362]:
def entrenarQ ( Q, R, Gamma, objetivo, num_episodios ):
    while num_episodios > 0:
        estado = random.randint ( 0, num_fil**2 - 1 )
        while ( estado != objetivo ):
            #acciones = acciones ( R, estado )
            accion_elegida = random.choice ( acciones ( R, estado ) )
            siguiente_estado = accion_elegida
            Q [ estado, accion_elegida ] = R [ estado, accion_elegida ] + Gamma * maxQValue(Q,R,siguiente_estado)
            estado = siguiente_estado
        num_episodios = num_episodios - 1
    return Q

In [347]:
maxQValue(qMatrix,rMatrix,2)

500.0

In [363]:
entrenarQ(qMatrix, rMatrix, 0.8, objetivo, 1000)

print(qMatrix)

[[  0  64  80  64   0   0   0   0   0]
 [ 64   0  80  64  64  80   0   0   0]
 [ 64  64   0  64  64  80 100  80  64]
 [ 64  64  80   0   0   0   0  80  64]
 [  0  64  80   0   0  80   0   0   0]
 [  0  64  80   0  64   0 100  80   0]
 [  0   0   0   0   0   0   0   0   0]
 [  0   0  80  64   0  80 100   0  64]
 [  0   0  80  64   0   0   0  80   0]]


In [364]:
qMatrix

array([[  0,  64,  80,  64,   0,   0,   0,   0,   0],
       [ 64,   0,  80,  64,  64,  80,   0,   0,   0],
       [ 64,  64,   0,  64,  64,  80, 100,  80,  64],
       [ 64,  64,  80,   0,   0,   0,   0,  80,  64],
       [  0,  64,  80,   0,   0,  80,   0,   0,   0],
       [  0,  64,  80,   0,  64,   0, 100,  80,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,  80,  64,   0,  80, 100,   0,  64],
       [  0,   0,  80,  64,   0,   0,   0,  80,   0]])

In [365]:
qMatrix=(qMatrix/np.max(qMatrix))*100
print(qMatrix)

[[  0.  64.  80.  64.   0.   0.   0.   0.   0.]
 [ 64.   0.  80.  64.  64.  80.   0.   0.   0.]
 [ 64.  64.   0.  64.  64.  80. 100.  80.  64.]
 [ 64.  64.  80.   0.   0.   0.   0.  80.  64.]
 [  0.  64.  80.   0.   0.  80.   0.   0.   0.]
 [  0.  64.  80.   0.  64.   0. 100.  80.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.  80.  64.   0.  80. 100.   0.  64.]
 [  0.   0.  80.  64.   0.   0.   0.  80.   0.]]


In [366]:
rMatrix

array([[ -1,   0,   0,   0,  -1,  -1,  -1,  -1,  -1],
       [  0,  -1,   0,   0,   0,   0,  -1,  -1,  -1],
       [  0,   0,  -1,   0,   0,   0, 100,   0,   0],
       [  0,   0,   0,  -1,  -1,  -1,  -1,   0,   0],
       [ -1,   0,   0,  -1,  -1,   0,  -1,  -1,  -1],
       [ -1,   0,   0,  -1,   0,  -1, 100,   0,  -1],
       [ -1,  -1,   0,  -1,  -1,   0, 100,   0,  -1],
       [ -1,  -1,   0,   0,  -1,   0, 100,  -1,   0],
       [ -1,  -1,   0,   0,  -1,  -1,  -1,   0,  -1]])

Camino eficiente
-----------------------------------------------------------

In [367]:
def findOptimalWay(initialState, Q, objetive):
    #paso1 
    actualState = initialState
    camino = [initialState]
    contador=0
    totalValue=0
    #paso2
    while(actualState!=objetive):
        qRow = Q[actualState]
        maxValue = np.max(qRow)
        maxValueIndex = np.where(qRow == maxValue)
        nextState=maxValueIndex[0][0]
        print("nextState",nextState)
        camino.append(nextState)
        #paso3
        totalValue += maxValue 
        actualState=nextState
        print(actualState)
        contador=contador+1
    return camino, contador, totalValue
    

In [368]:
[camino,contador,totalValue]=findOptimalWay(8, qMatrix, objetivo)

nextState 2
2
nextState 6
6


In [369]:
camino

[8, 2, 6]